<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

## Package Imports

In [4]:
import pandas as pd
import numpy as np
import re
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [5]:
import spacy.cli
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
nlp = spacy.load('en_core_web_sm')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
nltk.download('stopwords')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'nltk' is not defined

In [26]:
from nltk.corpus import stopwords

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
stopwords = stopwords.words('english')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
stopwords.extend(['from', 'subject', 're', 'edu', 'use'])

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Importing & Cleaning Data

In [9]:
df = pd.read_csv('data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df.head()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [11]:
Y = df['primaryCategories']

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
Y.nunique()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


9

In [13]:
X = df['reviews.text']

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
example_text = X[9]
example_text

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"AmazonBasics AA AAA batteries have done well by me appear to have a good shelf life. I'll buy them again."

In [15]:
def data_clean(doc):
    """
    Function to return clean data.
    """
    # remove everything except letter and numbers
    doc = re.sub('[^a-zA-Z 0-9]', '', doc)
    
    # lowercase everything
    doc = doc.lower()
    
    #return cleaned data
    return doc

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data_clean(example_text)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'amazonbasics aa aaa batteries have done well by me appear to have a good shelf life ill buy them again'

In [17]:
X = X.apply(lambda doc: data_clean(doc))

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
X.head()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    i order 3 of them and one of the item is bad q...
1    bulk is always the less expensive way to go fo...
2    well they are not duracell but for the price i...
3    seem to work as well as name brand batteries a...
4    these batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [19]:
# Analyzing targe data
Y.nunique()
Y.unique()
# All instances of target data are one of 9 distinct labels
# There is no need to clean the target data any more 

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Health & Beauty', 'Electronics', 'Office Supplies',
       'Animals & Pet Supplies', 'Home & Garden', 'Electronics,Furniture',
       'Toys & Games,Electronics', 'Electronics,Media',
       'Office Supplies,Electronics'], dtype=object)

## Tokenizing Data & Removing Stopwords

In [20]:
data = X.tolist()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Tokenize data using genism's built in preprecessor 

def tokenizer(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(sentence, deacc = True)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
data_tok = list(tokenizer(X))

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# Remove Stop Words

def remove_stopwords(texts):
    # return a list or words inside a list if it is not a stopword
    return [[word for word in doc if word not in stopwords] for doc in texts]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
def lemmitization(texts, allowed_pos = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    lem = []
    for word in texts:
        # rejoin individual words in each inner list
        doc = nlp(" ".join(word)) 
        # take the lemma of each word and return it back 
        # to it's list if it is an allowed grammar type (noun, adj...)
        lem.append([token.lemma_ for token in doc if token.pos_ in allowed_pos])
    return lem

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
no_stop = remove_stopwords(data_tok)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
data_lem = lemmitization(no_stop)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
data_lem

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['order',
  'item',
  'bad',
  'quality',
  'miss',
  'backup',
  'spring',
  'put',
  'pcs',
  'aluminum',
  'make',
  'battery',
  'work'],
 ['bulk', 'always', 'less', 'expensive', 'way', 'go', 'product'],
 ['duracell', 'price', 'happy'],
 ['seem', 'work', 'name', 'brand', 'battery', 'much', 'well', 'price'],
 ['battery', 'long', 'last', 'price', 'great'],
 ['buy',
  'lot',
  'battery',
  'christmas',
  'amazonbasic',
  'cell',
  'good',
  'notice',
  'difference',
  'brand',
  'name',
  'battery',
  'amazon',
  'basic',
  'brand',
  'lot',
  'easy',
  'purchase',
  'arrive',
  'house',
  'hand',
  'buy'],
 ['problame', 'battery', 'order', 'pleased'],
 ['well',
  'look',
  'cheap',
  'nonrechargeable',
  'battery',
  'last',
  'quite',
  'perfect',
  'say'],
 ['hold',
  'amount',
  'high',
  'power',
  'juice',
  'energizer',
  'duracell',
  'half',
  'price'],
 ['amazonbasic',
  'aa',
  'aaa',
  'battery',
  'do',
  'well',
  'appear',
  'good',
  'shelf',
  'life',
  'ill',
  'buy

In [138]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.lemma_)

Apple
be
look
at
buy
U.K.
startup
for
$
1
billion


/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Fitting a Genism LDA topic Model

In [151]:
# Creating library and corpus

# Create Dictionary
id2word = corpora.Dictionary(data_lem)

# Term Document Frequency for each doc -> doc-term matrix 
corpus = [id2word.doc2bow(list_of_token) for list_of_token in data_lem]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [152]:
corpus

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)],
 [(20, 1), (21, 1), (22, 1)],
 [(3, 1), (12, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)],
 [(3, 1), (22, 1), (28, 1), (29, 1), (30, 1)],
 [(3, 2),
  (23, 2),
  (25, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1)],
 [(3, 1), (7, 1), (46, 1), (47, 1)],
 [(3, 1),
  (27, 1),
  (29, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(20, 1),
  (22, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1)],
 [(3, 1),
  (27, 1),
  (32, 1),
  (35, 1),
  (40, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1)],
 [(3, 1),
  (22, 1),
  (23, 1),
  (25, 1),
  (28, 1),
  (31, 1),
  (34, 1),
  (35, 

In [157]:
NUM_TOPICS = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                           num_topics = NUM_TOPICS, 
                                           id2word=id2word, 
                                           passes=15)
ldamodel.save('model5.gensim')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [158]:
pprint(ldamodel.print_topics())

[(0,
  '0.103*"great" + 0.072*"price" + 0.064*"product" + 0.052*"good" + '
  '0.029*"recommend" + 0.029*"amazon" + 0.025*"buy" + 0.019*"work" + '
  '0.016*"excellent" + 0.016*"quality"'),
 (1,
  '0.107*"love" + 0.049*"old" + 0.045*"buy" + 0.042*"read" + 0.039*"year" + '
  '0.039*"book" + 0.035*"easy" + 0.023*"gift" + 0.023*"purchase" + '
  '0.020*"daughter"'),
 (2,
  '0.079*"kindle" + 0.026*"buy" + 0.025*"fire" + 0.019*"charge" + 0.018*"go" + '
  '0.017*"get" + 0.015*"use" + 0.014*"new" + 0.014*"time" + 0.012*"day"'),
 (3,
  '0.141*"app" + 0.136*"good" + 0.031*"wife" + 0.031*"price" + 0.026*"child" + '
  '0.024*"fast" + 0.022*"user" + 0.020*"storage" + 0.018*"deal" + 0.017*"buy"'),
 (4,
  '0.146*"tablet" + 0.043*"kid" + 0.034*"great" + 0.033*"game" + 0.030*"fire" '
  '+ 0.025*"play" + 0.025*"amazon" + 0.016*"well" + 0.015*"get" + '
  '0.012*"video"'),
 (5,
  '0.017*"amazon" + 0.015*"need" + 0.014*"model" + 0.013*"make" + 0.011*"use" '
  '+ 0.011*"birthday" + 0.011*"come" + 0.011*"get" 

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [159]:
Y.unique()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Health & Beauty', 'Electronics', 'Office Supplies',
       'Animals & Pet Supplies', 'Home & Garden', 'Electronics,Furniture',
       'Toys & Games,Electronics', 'Electronics,Media',
       'Office Supplies,Electronics'], dtype=object)

In [163]:
# Compute Coherence Score
coherence_model_ldamodel = CoherenceModel(model=ldamodel, texts=data_lem, dictionary=id2word, coherence='c_v')
coherence_ldamodel = coherence_model_ldamodel.get_coherence()
print('\nCoherence Score: ', coherence_ldamodel)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.4626648667987408


In [164]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -6.820239311952759


In [165]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, id2word)
vis

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.041254  0.150477       1        1  16.725793
4     -0.015228  0.155703       2        1  15.891125
1      0.001635  0.173788       3        1  14.510130
7      0.223567 -0.083138       4        1  14.210501
0      0.225265 -0.008018       5        1  11.068792
5     -0.092894  0.143408       6        1   9.837998
8     -0.229755 -0.040591       7        1   6.779134
3      0.187327 -0.211850       8        1   6.117140
6     -0.258664 -0.279779       9        1   4.859386, topic_info=         Term         Freq        Total Category  logprob  loglift
3     battery  7592.000000  7592.000000  Default  30.0000  30.0000
3879   tablet  7865.000000  7865.000000  Default  29.0000  29.0000
40       good  6887.000000  6887.000000  Default  28.0000  28.0000
95       love  5762.000000  5762.000000  Default  27.0000  27.0000
1822      app  2929.000000  2929.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
406      take   325.686056  1139.084463   Topic9  -3.9253   1.7722
148        tv   119.069506   223.663108   Topic9  -4.9316   2.3938
264    update   112.448277   213.351627   Topic9  -4.9888   2.3838
798     sound   108.273142   329.019509   Topic9  -5.0266   1.9128
2378    cover   107.437766   312.031312   Topic9  -5.0344   1.9581

[493 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
61        1  0.006530       aa
61        6  0.992515       aa
62        4  0.875939      aaa
62        6  0.121365      aaa
944       3  0.992204  ability
...     ...       ...      ...
106       2  0.006306     year
106       3  0.867925     year
1063      8  0.997374    young
3205      9  0.995396  youtube
2211      6  0.994941       yr

[713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 8, 1, 6, 9, 4, 7])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling